<a href="https://colab.research.google.com/github/jai-llm/TEXT2SQL/blob/main/3_Text2SQL_HFUpload_Llama2GPTQFineTunev3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Text2SQL Upload LLaMA2GPTQ Fine-Tune to HF
Runs in Free Google Colab needs T4 GPU to run.

In [7]:
# !pip install -q -U datasets
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes
# ! pip install -q -U rouge_score
# ! pip install -q -U langchain
!pip install -q -U huggingface_hub
# !pip install -q -U dotenv

### Imports

In [8]:
import pandas as pd
import json
import torch
import os
import time

# In case Login Required For Model
from huggingface_hub import login
# from dotenv import load_dotenv

from datasets import load_dataset, Dataset, load_metric, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print

# LangChain Imports
# from langchain.llms import HuggingFacePipeline
# from langchain import PromptTemplate  #, LLMChain

# Imports for QA Retrieval Chain
# from langchain.chains import RetrievalQA

# Import to Cleanup LLM Output
# import textwrap

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


### Global Constants

In [9]:
model_id = "TheBloke/Llama-2-7B-GPTQ"
# model_id = "TheBloke/Llama-2-7b-Chat-GPTQ"
MODEL_PATH = "/content/drive/MyDrive/Text2SQL/FTClean/"
MODEL_DIR = "sql_gptq_training"+"_30/"
checkpoint_name ="SQL_llama2_gptq_7b_peftv1_20230930_043049"

In [10]:
print(checkpoint_name)

SQL_llama2_gptq_7b_peftv1_20230930_043049

In [11]:
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"
LLMRES_PKL_FILE="sql_test_llmres.pkl"

In [12]:
HUGGINGFACE_TOKEN = ""

### Common Functions

In [13]:
def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

In [14]:
def gen_eval_res(test_df):
  eval_res = []
  time_res = []
  row_no = 1
  # tmp = test_df.iloc[:2]
  for row in test_df.itertuples(index=False):
    start_time = perf_counter()
    # print(row.text)
    text = row.text # test_df['text'][ID]
    llm_res = pipe(text)[0]['generated_text']
    parse_res = parse(llm_res)
    end_time = perf_counter()
    eval_res.append(parse_res)
    time_res.append(end_time-start_time)
    print(f"*** Row {row_no} Done ***")
    row_no += 1

  # test_df['eval_res'] = eval_res
  # test_df['eval_time'] = time_res
  display(pd.DataFrame(eval_res))
  display(pd.DataFrame(time_res))

### Hugging Face Login Setup

In [15]:
# If Login Required for Model Access
# load_dotenv("/notebooks/.env")
os.environ["TOKENIZERS_PARALLELISM"]="false"
login(token=HUGGINGFACE_TOKEN)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [16]:
# !huggingface-cli login

### Load Test Data

In [17]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

### Load Model and Setup Trainer

In [18]:
output_dir = MODEL_PATH + MODEL_DIR + checkpoint_name
print(output_dir)

/content/drive/MyDrive/Text2SQL/FTClean/sql_gptq_training_30/SQL_llama2_gptq_7b_peftv1_20230930_043049

In [19]:
# Tokenizer - Base Model since we did not change it
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# LLM GPTQ Fine-Tune Model
# To perform inference on the test dataset
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
model = AutoPeftModelForCausalLM.from_pretrained(
          output_dir,
          low_cpu_mem_usage=True,
          return_dict=True,
          torch_dtype=torch.float16,
          device_map="auto" #"cuda", # Forces model to use GPU
      )

RuntimeError: ignored

In [ ]:
# Get Model Memory Footprint = ~4GB
print(model.get_memory_footprint()/1e9) # GB

3.997253632

### Save Model to HuggingFace Hub

In [ ]:
# !huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

### Pull the Model From Hub

In [ ]:
MODEL_ID = ""
# Tokenizer - Base Model since we did not change it
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# LLM GPTQ Fine-Tune Model
# To perform inference on the test dataset
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
model = AutoPeftModelForCausalLM.from_pretrained(
          model_id=MODEL_ID,
          low_cpu_mem_usage=True,
          return_dict=True,
          torch_dtype=torch.float16,
          device_map="cuda", # Forces model to use GPU
      )

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=180,
    do_sample = True,
    temperature=0.1,
    top_k=4,
    # top_p=0.95,
    repetition_penalty=1.2
)
#  penalty_alpha=0.5,
#     # do_sample = True,
#     top_k=1,
#     # temperature=0.1,
#     repetition_penalty=1.2,
#     max_new_tokens=180

llm = HuggingFacePipeline(pipeline=pipe)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'Refor

#### Check Model on Sample Test Question

In [ ]:
ID = 10

In [ ]:
test_df['text'][ID]

'### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    Show ids for all students who have advisor 1121.\n    ### Context:\n    CREATE TABLE Student (StuID VARCHAR, Advisor VARCHAR)\n    ### Response:\n    '

In [ ]:
display(test_df['response'][ID])

'SELECT StuID FROM Student WHERE Advisor = 1121'

In [ ]:
text = test_df['text'][ID]
llm_res = pipe(text)[0]['generated_text']
parse_res = parse(llm_res)
# print(llm_res)
print(parse_res)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


SELECT StuID FROM Student WHERE Advisor = "1121"

### Get LLM Responses For All Questions

In [ ]:
eval_start_time = perf_counter()
gen_eval_res(test_df.iloc[:4]) # test_df.iloc[:4]
eval_end_time = perf_counter()
print('Evaluation Took: ', round(eval_end_time-eval_start_time, 2))

*** Row 1 Done ***

*** Row 2 Done ***

*** Row 3 Done ***

*** Row 4 Done ***

*** Row 5 Done ***

...

*** Row 451 Done ***

*** Row 452 Done ***

*** Row 453 Done ***

*** Row 454 Done ***

Evaluation Took:  2226.14